In [1]:
import sys
sys.path.append('/mnt/home/tf_transformers2/')

In [ ]:
# Latest versions have name changed from nlp -> datasets
# pip install datasets
from nlp import load_dataset
from transformers import BertTokenizer
import numpy as np
from tf_transformers.utils import TFProcessor

import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)



In [ ]:
dataset = load_dataset('squad')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def parse_squad(dataset, tokenizer, context_limit=64, max_counter=10000):
    for counter,entry in enumerate(dataset):
        answer = entry['answers']['text'][0]
        context_ids = tokenizer.encode(entry['context'])
        if len(context_ids) < context_limit:
            diff = context_limit - len(context_ids)
            context_ids = context_ids + [0] * diff
        id_ = entry['id']
        question_ids = tokenizer.encode(entry['question'])
        title_tokens = entry['title'].split('_')
        
        result = {}
        result['answer'] = answer
        result['context_ids'] = context_ids[:context_limit]
        result['id'] = id_
        result['question_ids'] = question_ids
        result['title_tokens'] = title_tokens
        result['dummy_classification_label']  = np.random.randint(0, 10)
        result['dummy_float_id'] = np.random.randint(0,10000) * 1.0 # make it float
        
        if counter < max_counter:
            yield result
        else:
            break

In [ ]:
tfprocessor = TFProcessor()
train_parser = parse_squad(dataset['train'], tokenizer)
# Returns a tf.data.Dataset
dataset = tfprocessor.process(train_parser)
# Lets batch it ( we dont need to pad things here, as we deal with amazing tf.RaggedTensor)
dataset = dataset.batch(5)